########## choose stock list (aka market) ##################

In [25]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date

In [26]:
## To acquire stock list in S&P500 ###
ex1 = pd.read_csv('stock_info_s&p500.csv')
df = pd.DataFrame(ex1['Symbol'].astype('str'))
df = df.rename(columns={'Symbol' : 'ticker'})
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df

,ticker
0,AAPL
1,MSFT
2,NVDA
3,GOOGL
4,GOOG
...,...
498,BBWI
499,BWA
500,NCLH
501,AAL


### Acquire informations from yfinance function >> .info

In [27]:
# 1st pulling date for this data #
df['date_pulling'] = date.today()

In [28]:
# 2nd acquiring information from .info #
info_attribute_list = [
    'industry',
    'sector',
    'enterpriseValue',
    'totalCashPerShare',
    'profitMargins',
    'trailingPE'
]
# 3rd acquiring information from .quarterly_balance_sheet #
balancesheet_list = [
    'Total Assets',
    'Current Liabilities'
]

# 4th acquiring information from .quarterly_financials #
ttm = 4
financials_list = [
    'EBIT',
    'Operating Income'
]

info_attribute_list_buffer = []
balancesheet_list_buffer = []
financials_list_buffer = []
for i in range(len(df)):
    # print(i, " ",df['ticker'][i])
    yfticker = yf.Ticker(df['ticker'][i])

    # 2nd acquiring information from .info #
    info_attribute_list_buffer_r = []
    for j in range(len(info_attribute_list)):
        try:
            info_attribute_list_buffer_r.append(yfticker.info[info_attribute_list[j]])
        except:
            info_attribute_list_buffer_r.append(None)
    info_attribute_list_buffer.append(info_attribute_list_buffer_r)

    # 3rd acquiring information from .balancesheet #
    balancesheet_list_buffer_r = []
    for j in range(len(balancesheet_list)):
        try:
            balancesheet_list_buffer_r.append(yfticker.quarterly_balance_sheet.loc[yfticker.quarterly_balance_sheet.index == balancesheet_list[j]].values[0][0])
        except:
            balancesheet_list_buffer_r.append(None)
    balancesheet_list_buffer.append(balancesheet_list_buffer_r)

    # 4th acquiring information from .quarterly_financials #
    financials_list_buffer_r = []
    for j in range(len(financials_list)):
        try:
            financials_list_buffer_r.append(np.array([yfticker.quarterly_financials.loc[yfticker.quarterly_financials.index == financials_list[j]].values[0][i] for i in range(ttm)]).sum())
        except:
            financials_list_buffer_r.append(None)
    try:
        financials_list_buffer_r.append(yfticker.quarterly_financials.columns[0])
    except:
        financials_list_buffer_r.append(None)
    financials_list_buffer.append(financials_list_buffer_r)

financials_list.append('ttm_latest')
df = df.join(pd.DataFrame(info_attribute_list_buffer, columns=info_attribute_list))
df = df.join(pd.DataFrame(balancesheet_list_buffer, columns=balancesheet_list))
df = df.join(pd.DataFrame(financials_list_buffer, columns=financials_list))


#### Drop NAN #############
df = df.dropna()
##### Drop where marketcap and EBIT < 0#############
df = df.loc[df['enterpriseValue'] >0]
df = df.loc[df['EBIT'] >0]
df = df.loc[df['Operating Income'] >0]
df = df.reset_index(drop=True)

df.to_csv('data_stock_s&p500.csv',index= False)

############################################################################

In [29]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
df = pd.read_csv('data_stock_s&p500.csv')
print('sector', df['sector'].unique())
print('industry', df['industry'].unique())

sector ['Technology' 'Communication Services' 'Consumer Cyclical' 'Healthcare'
 'Consumer Defensive' 'Energy' 'Financial Services' 'Basic Materials'
 'Industrials' 'Utilities' 'Real Estate']
industry ['Consumer Electronics' 'Software - Infrastructure' 'Semiconductors'
 'Internet Content & Information' 'Internet Retail'
 'Drug Manufacturers - General' 'Auto Manufacturers' 'Discount Stores'
 'Healthcare Plans' 'Oil & Gas Integrated' 'Credit Services'
 'Household & Personal Products' 'Home Improvement Retail'
 'Beverages - Non-Alcoholic' 'Entertainment' 'Software - Application'
 'Diagnostics & Research' 'Telecom Services' 'Specialty Chemicals'
 'Information Technology Services' 'Restaurants' 'Communication Equipment'
 'Medical Devices' 'Aerospace & Defense' 'Tobacco'
 'Semiconductor Equipment & Materials' 'Medical Instruments & Supplies'
 'Farm & Heavy Construction Machinery' 'Utilities - Regulated Electric'
 'Financial Data & Stock Exchanges' 'Railroads' 'Biotechnology'
 'Asset Managemen

In [30]:

### Next I perform calculations for MF_ROC and MF_EY ###
# represents_earning = 'Operating Income'
represents_earning = 'EBIT'

df['MF_ROC'] = df[represents_earning]/(df['Total Assets'] - df['Current Liabilities'])
df['MF_EY'] = df[represents_earning]/df['enterpriseValue']
df

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY
0,AAPL,2024-08-20,Consumer Electronics,Technology,3.473966e+12,4.065,0.26441,34.382040,3.316120e+11,1.316240e+11,1.216250e+11,1.205940e+11,2024-06-30,0.608161,0.035010
1,MSFT,2024-08-20,Software - Infrastructure,Technology,3.155570e+12,10.162,0.35956,35.722880,5.121630e+11,1.252860e+11,1.107220e+11,1.094330e+11,2024-06-30,0.286194,0.035088
2,NVDA,2024-08-20,Semiconductors,Technology,3.177799e+12,1.278,0.53398,76.470590,7.707200e+10,1.522300e+10,4.914300e+10,4.774000e+10,2024-04-30,0.794564,0.015464
3,GOOGL,2024-08-20,Internet Content & Information,Communication Services,1.979702e+12,8.182,0.26702,23.912483,4.147700e+11,7.791300e+10,1.018210e+11,9.793700e+10,2024-06-30,0.302268,0.051432
4,GOOG,2024-08-20,Internet Content & Information,Communication Services,2.000998e+12,8.182,0.26702,24.160688,4.147700e+11,7.791300e+10,1.018210e+11,9.793700e+10,2024-06-30,0.302268,0.050885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,FMC,2024-08-20,Agricultural Inputs,Basic Materials,1.201035e+10,3.777,0.33968,5.572649,1.213070e+10,3.523100e+09,2.377000e+08,5.647000e+08,2024-06-30,0.027615,0.019791
408,BBWI,2024-08-20,Specialty Retail,Consumer Cyclical,1.225191e+10,3.830,0.11919,8.852713,5.221000e+09,1.221000e+09,1.365000e+09,1.292000e+09,2024-04-30,0.341250,0.111411
409,BWA,2024-08-20,Auto Parts,Consumer Cyclical,1.031260e+10,5.655,0.04971,9.933935,1.410800e+10,3.712000e+09,1.238000e+09,1.261000e+09,2024-06-30,0.119084,0.120047
410,NCLH,2024-08-20,Travel Services,Consumer Cyclical,2.061598e+10,1.351,0.04624,16.639174,2.011383e+10,6.633916e+09,1.204430e+09,1.207603e+09,2024-06-30,0.089350,0.058422


In [31]:
### Drop Utilities, Energy, and Financial Services  as suggested from the book ###
sectortoexclude = [
    'Utilities',
    'Energy',
    'Financial Services',
    'Real Estate'
]
for i in sectortoexclude:
    print('exclude sector > ',i)
    try:
         
        df = df.loc[df['sector'] != i]
    except:
        None
        
industrytoexclude =[
    'Engineering & Construction',
    'Building Products & Equipment',
    'Building Materials'
]
for i in industrytoexclude:
    print('exclude industry > ',i)
    try:
         
        df = df.loc[df['industry'] != i]
    except:
        None

df = df.reset_index(drop=True)

exclude sector >  Utilities
exclude sector >  Energy
exclude sector >  Financial Services
exclude sector >  Real Estate
exclude industry >  Engineering & Construction
exclude industry >  Building Products & Equipment
exclude industry >  Building Materials


In [32]:
#### Select market size by choosing ' market ' ###
market =  50000000### in USD
df_market = df.loc[df['enterpriseValue'] >= market]
df_market = df_market.reset_index(drop=True)

In [33]:
### Ranking regarding MFs####

df_market['Ranking_MF_ROC'] = df_market['MF_ROC'].rank()
df_market['Ranking_MF_EY'] = df_market['MF_EY'].rank()
df_market['Ranking_MF'] = df_market['Ranking_MF_ROC'] + df_market['Ranking_MF_EY']

df_market.loc[df_market['Ranking_MF'] == df_market['Ranking_MF'].max()]

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
75,MO,2024-08-20,Tobacco,Consumer Defensive,1.108374e+11,1.054,0.50292,8.848275,3.438700e+10,7.782000e+09,1.467600e+10,1.140900e+10,2024-06-30,0.551626,0.13241,296.0,296.0,592.0


In [34]:
numstocks = 50
df_sorted = df_market.sort_values(by=['Ranking_MF'],ascending=False)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted[:numstocks]

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,MO,2024-08-20,Tobacco,Consumer Defensive,1.108374e+11,1.054,0.50292,8.848275,3.438700e+10,7.782000e+09,1.467600e+10,1.140900e+10,2024-06-30,0.551626,0.132410,296.0,296.0,592.0
1,JBL,2024-08-20,Electronic Components,Technology,1.296782e+10,21.887,0.04625,9.760218,1.744900e+10,1.142300e+10,1.990000e+09,1.520000e+09,2024-05-31,0.330236,0.153457,271.0,302.0,573.0
2,BBWI,2024-08-20,Specialty Retail,Consumer Cyclical,1.225191e+10,3.830,0.11919,8.852713,5.221000e+09,1.221000e+09,1.365000e+09,1.292000e+09,2024-04-30,0.341250,0.111411,275.0,288.0,563.0
3,ULTA,2024-08-20,Specialty Retail,Consumer Cyclical,1.936010e+10,10.994,0.11126,14.728906,5.631939e+09,1.572724e+09,1.647678e+09,1.647678e+09,2024-04-30,0.405911,0.085107,284.0,272.0,556.0
4,PHM,2024-08-20,Residential Construction,Consumer Cyclical,2.685827e+10,6.723,0.16751,9.563025,1.654407e+10,3.290047e+09,3.711561e+09,3.607165e+09,2024-06-30,0.280033,0.138191,252.0,299.0,551.0
5,NVR,2024-08-20,Residential Construction,Consumer Cyclical,2.585818e+10,792.331,0.16183,18.315140,6.407288e+09,1.260029e+09,2.055058e+09,2.050196e+09,2024-06-30,0.399253,0.079474,283.0,264.0,547.0
6,EBAY,2024-08-20,Internet Retail,Consumer Cyclical,3.072752e+10,10.564,0.26408,11.027238,2.037800e+10,5.799000e+09,3.905000e+09,2.045000e+09,2024-06-30,0.267851,0.127085,249.0,295.0,544.0
7,HPQ,2024-08-20,Computer Hardware,Technology,4.216916e+10,2.572,0.05602,11.689189,3.743300e+10,2.483900e+10,3.909000e+09,4.305000e+09,2024-04-30,0.310386,0.092698,267.0,275.0,542.0
8,LULU,2024-08-20,Apparel Retail,Consumer Cyclical,3.189201e+10,15.205,0.16090,20.796146,6.828495e+09,1.383571e+09,2.238405e+09,2.238405e+09,2024-04-30,0.411099,0.070187,285.0,249.0,534.0
9,PAYC,2024-08-20,Software - Application,Technology,8.541613e+09,6.194,0.26550,19.190820,4.312744e+09,2.526883e+09,6.051910e+08,5.845780e+08,2024-06-30,0.338879,0.070852,273.0,254.0,527.0


In [35]:
df_sorted.loc[df_sorted['ticker'] == 'NVO']

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF


In [36]:
df_sorted_2 = df_sorted[:numstocks].copy()
df_sorted_2 = df_sorted_2.sort_values(by=['totalCashPerShare'],ascending=False)
df_sorted_2 = df_sorted_2.reset_index(drop=True)
df_sorted_2

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,NVR,2024-08-20,Residential Construction,Consumer Cyclical,2.585818e+10,792.331,0.16183,18.315140,6.407288e+09,1.260029e+09,2.055058e+09,2.050196e+09,2024-06-30,0.399253,0.079474,283.0,264.0,547.0
1,BKNG,2024-08-20,Travel Services,Consumer Cyclical,1.251176e+11,487.019,0.22456,25.973549,2.854100e+10,1.820600e+10,7.364000e+09,6.357000e+09,2024-06-30,0.712530,0.058857,300.0,220.0,520.0
2,MOH,2024-08-20,Healthcare Plans,Healthcare,1.396915e+10,148.481,0.02946,18.979166,1.518500e+10,7.836000e+09,1.535000e+09,1.535000e+09,2024-06-30,0.208872,0.109885,217.0,287.0,504.0
3,DE,2024-08-20,Farm & Heavy Construction Machinery,Industrials,1.611468e+11,26.080,0.14698,12.722961,1.056280e+11,3.928400e+10,1.505400e+10,1.408000e+10,2024-04-30,0.226908,0.093418,228.0,276.0,504.0
4,SNA,2024-08-20,Tools & Accessories,Industrials,1.468396e+10,23.397,0.20218,14.408311,7.759900e+09,9.501000e+08,1.402400e+09,1.337100e+09,2024-06-30,0.205939,0.095506,215.0,279.0,494.0
5,NUE,2024-08-20,Steel,Basic Materials,3.684848e+10,22.896,0.10451,10.387167,3.418522e+10,4.865261e+09,5.095961e+09,4.638359e+09,2024-06-30,0.173805,0.138295,185.0,300.0,485.0
6,JBL,2024-08-20,Electronic Components,Technology,1.296782e+10,21.887,0.04625,9.760218,1.744900e+10,1.142300e+10,1.990000e+09,1.520000e+09,2024-05-31,0.330236,0.153457,271.0,302.0,573.0
7,CPAY,2024-08-20,Software - Infrastructure,Technology,2.645872e+10,20.272,0.26413,21.410313,1.619654e+10,7.842547e+09,1.707174e+09,1.699705e+09,2024-06-30,0.204354,0.064522,213.0,235.0,448.0
8,AZO,2024-08-20,Specialty Retail,Consumer Cyclical,6.646121e+10,19.669,0.14604,21.978659,1.710843e+10,9.192587e+09,3.726577e+09,3.714523e+09,2024-05-31,0.470774,0.056071,291.0,210.0,501.0
9,GWW,2024-08-20,Industrial Distribution,Industrials,4.904586e+10,15.750,0.10858,26.243223,8.352000e+09,2.397000e+09,2.570000e+09,2.542000e+09,2024-06-30,0.431570,0.052400,287.0,197.0,484.0
